In [1]:
import os, sys
import numpy as np
from collections import Counter
sys.path.append('../readmaroc')
from maroc_data_new_tboard import MarocData
from operator import add
from functools import reduce

In [2]:
input_dat = '../readmaroc/data/farm/Run000109218.dat'

In [3]:
y_offset = [12000, 10000,8000, 4000, 2000]

In [4]:
marocs = [(i, j) for i, j in zip(np.arange(0,384,64), np.arange(0,384,64)[1:])]

In [5]:
marocdata = MarocData(input_dat)

In [6]:
marocdata.fix_p1(debug=False)

In [128]:
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.plotting import reset_output
from bokeh.layouts import gridplot
from bokeh.models import Span, Text, ColumnDataSource

In [131]:
def board_plot(fig, ts, marocdata, board_id, board_idx, triplet_idx, c='blue'):
    if board_id in marocdata.active_boards:
        board = marocdata.get_board(board_id)
        if ts in board.clean_timestamps.keys():
            evt = board.clean_timestamps[ts]
            if evt in board:
                signal = board.signals[evt]
                pedestal = board.avg_data
                noise = board.noise
                if np.any(signal-pedestal>noise):
                    over = np.where(signal-pedestal>noise)[0]
                    seed = np.max((signal-pedestal)[over])
                    over_x = np.where(signal-pedestal==seed)[0][0]
                    fig.circle(over_x+ (board_idx * 320),
                        seed*2 + y_offset[triplet_idx],
                        color='black',
                        size=2,
                    )
                fig.line(np.arange(0 + (board_idx * 320), 320 + 320 * board_idx),
                (signal-pedestal)*2 + y_offset[triplet_idx],
                color=c,
                line_width=1)
                    
                rpc_division =Span(location= 320 * (board_idx + 1),
                    dimension='height', line_color='gray',
                    line_dash='dashed', line_width=1)
                fig.add_layout(rpc_division)

                board_text = Text(x='x', y='y', text='text')
                sourcedict = ColumnDataSource(dict(x=
                (320 + 320 * board_idx - 0 + (board_idx * 320)) / 2 - 150,
                y=y_offset[triplet_idx] - 300,
                text="board: {}".format(board_id) 
                                                  ))
                fig.add_glyph(sourcedict, board_text)
    return fig
                   

In [51]:
reset_output()

In [52]:
output_notebook()

Loading BokehJS ...

In [101]:
def plot_event_ts_new(ts, marocdata):
    print(ts)
    #fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 10), sharey=True, sharex=True)
    fig1=figure(width=300, height=350)
    fig2=figure(y_range = fig1.y_range, width=300, height=350)
    fig1.xgrid.grid_line_color = None
    fig1.ygrid.grid_line_color = None
    fig2.xgrid.grid_line_color = None
    fig2.ygrid.grid_line_color = None
    evt = None
    for i, (triplet_y, triplet_x) in enumerate(
        zip(np.arange(1, 16).reshape(5, 3), np.arange(16, 31).reshape(5, 3))
    ):
        for j, (board_y, board_x) in enumerate(zip(triplet_y, triplet_x)):
            if board_y in marocdata.active_boards:
                yboard = marocdata.get_board(board_y)
                ax1 = board_plot(fig1, ts, marocdata, board_y, j, i)
            if board_x in marocdata.active_boards:
                xboard = marocdata.get_board(board_x)
                ax2 = board_plot(fig2, ts, marocdata, board_x, j, i, c="red")
    #    if (evt is None):
    #        return None # skip
    #ax1.set_title("y layers", size="x-large")
    #ax2.set_title("x layers", size="x-large")
    #plt.yticks(
    #    y_offset,
    #    ["layer 0", "layer 1", "layer 2", "layer 3", "layer 4"],
    #    size="x-large",
    #)
#     plt.xticks([0, 320, 640, 960])
#     fig.text(0.5, 0.05, "strips", size="large")
#     fig.text(.5, .95, 'TS {}'.format(ts), size='large')
#     plt.ylim(1000, 13000)
#     plt.xlim(-10, 970)
    return fig1, fig2, ax1, ax2

In [54]:
def over_threshold_per_board(marocdata, pedestals, noise_tot):
    over_threshold_per_board = {}
    over_threshold_pos_per_board = {}
    for bid in marocdata.active_boards:
        timestamps = []
        board = marocdata.get_board(bid)
        for eid, signal in board.signals.items():
            if np.any((signal-pedestals[bid])>noise_tot[bid]):
                if len(np.where((signal-pedestals[bid])>noise_tot[bid])[0])<=60:
#                 print(bid, eid, np.where((signal-pedestals[bid])>noise_tot[bid]))
#                 plt.plot(np.arange(320), signal)
#                 plt.ylim(0,2000 if np.max(signal)<2000 else np.max(signal))
#                 plt.show()
                    event = board.get_event(eid)
                    timestamps.append(event.TS_norm)
        over_threshold_per_board[bid] = timestamps 
    return over_threshold_per_board

In [55]:
pedestals = marocdata.pedestals_tot

In [56]:
noise_tot = marocdata.noise_tot

In [37]:
over_threshold_per_board = over_threshold_per_board(marocdata, pedestals, noise_tot)

In [39]:
all_ts = reduce(add, over_threshold_per_board.values())

In [40]:
ts_to_plot = [ts for ts, occ in Counter(all_ts).items() if occ >= 2]

In [77]:
for ts in ts_to_plot:
    fig1, fig2, ax1, ax2 = plot_event_ts_new(ts, marocdata)
    grid = gridplot([fig1, fig2],ncols=2, width=250, height=250)

935032222
580169394
1483173091
1842514420
292674746
1439515936
285460945
1478958564
2134560940
491941217
37567627
1623713375
402159538
532090113
1966712151
543531103


In [132]:
fig1, fig2, ax1, ax2 = plot_event_ts_new(1842514420, marocdata)
grid = gridplot([fig1, fig2],ncols=2, width=400, height=500)
show(grid)

1842514420


ValueError: failed to validate ColumnDataSource(id='231174', ...).data: expected an element of ColumnData(String, Seq(Any)), got {'y': 11700, 'x': 10.0, 'text': 'board: 1'}